# Adversal validation
1. Isna by axis=0\
\
First import the essentials

In [26]:
import os
import sys
import keras
import tensorflow as tf
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/packages')
    !pip install git+https://github.com/keras-team/keras-tuner.git
    !pip install autokeras
    config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} )
    sess = tf.Session(config=config)
    keras.backend.set_session(sess)
else:
    path_fill = 'data/2022/filled_trian.csv'
    path_raw_train='data/2022/train.csv'
    path_A='data/2022/test_A榜.csv'
    path_B='data/2022/test_B榜.csv'

In [27]:
import pandas as pd
import useful_functions as uf
import autokeras as ak
df_train = pd.read_csv(path_raw_train).head(500)
df_train_filled = pd.read_csv(path_fill,index_col=0).head(500)
df_test_A = pd.read_csv(path_A).head(500)
df_test_B = pd.read_csv(path_B).head(500)

In [28]:
import numpy as np
import sklearn_pandas as sp
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

2. Drop too few columns, optional drop index for train data

In [29]:
def drop_too_few_variable(dataframe: pd.DataFrame, threshold=0.75, whether_index = False):
    total_rows = len(df_test_A.index)
    mask_drop = (dataframe.count() / total_rows ) > threshold
    dataframe = dataframe.loc[:, mask_drop]
    if whether_index:
        drop_index = dataframe.isna().sum(axis=1)>=len(dataframe.columns)*threshold
        drop_index = [ind for ind in drop_index.index if drop_index[ind]==True]
        dataframe.drop(index = drop_index)
        return dataframe.drop(index = drop_index)
    else:
        return dataframe
df_train = drop_too_few_variable(df_train,0.75,True)
portfolio:list = [df_test_A,df_test_B]
for i in range(len(portfolio)):
    portfolio[i] = drop_too_few_variable(portfolio[i])

3. step 1, minus 2

In [30]:
s_col = uf.dynamic_string_col(df_train)
df_train,string_col = uf.step1_data_processing(df_train,s_col)

# Need try iqr first or fillna first
\
Slip into two, nu and not nu
\
4.  Fillna, try random_forest, KNN, mode
## 3.2 数据集中缺失值从少到多进行排序
从缺失值最少的开始填，所以首先需要一个排序，即数据集中缺失值从少到多的一个顺序
# 3.3构建新特征矩阵和新标签

### Fill the string col first, fill with mode, then fill with iteraimpute

In [36]:
s_col = uf.dynamic_string_col(df_train)
df_train_str = df_train.loc[:,s_col]
impMode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df_train_str = \
    pd.DataFrame(impMode.transform(df_train_str),columns=s_col,index=df_train_str.index)

(498, 4)


(498, 4)

In [ ]:

sor_by_nan = df_nu.isna().sum().sort_values(ascending=True)

con = sor_by_nan>0
sor_by_nan =sor_by_nan[con].index # index

i=0    # loop over all columns with nan 
to_be_fillc:str = sor_by_nan[i]
x_tobe_0 = sor_by_nan[sor_by_nan!=to_be_fillc]

y_tobe_trained = df_train[df_train[to_be_fillc].notna()][to_be_fillc]
tobe_tested_index = df_train[df_train[to_be_fillc].isna()][to_be_fillc].index

x_tobe_tested = uf.get_numerical_df(df_train.loc[tobe_tested_index,x_tobe_0])
x_tobe_trained = uf.get_numerical_df(df_train.loc[y_tobe_trained.index,x_tobe_0])


#---------------- FILL IN ALL ZERO x train --------------------------------
imp0 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
x_tobe_trained=imp0.fit_transform(x_tobe_trained)
x_tobe_tested=imp0.transform(x_tobe_tested)


In [37]:
df_nu = uf.get_numerical_df(df_train)
sor_by_nan = df_nu.isna().sum().sort_values(ascending=True)
# star with the least missing number
con = sor_by_nan>0 # Filter out those with no missing values
sor_by_nan =sor_by_nan[con].index # index
i=0    # loop over all columns with nan
to_be_fillc:str = sor_by_nan[i]
x_tobe_0 = sor_by_nan[sor_by_nan!=to_be_fillc]
# columns names that are going to be filled woth zero as training data
